In [21]:
import os

In [22]:
%pwd

'd:\\End-to-End-MLProject-using-MLflow-and-EC2-Insance'

In [33]:
os.chdir("End-to-End-MLProject-using-MLflow-and-EC2-Insance")

In [34]:
%pwd

'd:\\End-to-End-MLProject-using-MLflow-and-EC2-Insance'

In [44]:
os.environ["MLFLOW_TRACKING_URL"] = "https://dagshub.com/atharvasamel/End-to-End-MLProject-using-MLflow-and-EC2-Insance.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"] = "atharvasamel"
os.environ["MLFLOW_TRACKING_PASSOWRD"] = "c120563b5e788a67d9d7b073e4f175fbafe4a4b3"

In [45]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    model_path: Path
    all_params: dict
    metric_file_name: Path
    target_column: str
    mlflow_uri: str

In [46]:
from mlopsWine.constants import *
from mlopsWine.utils.common import read_yaml, create_directories, save_json

In [47]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        params = self.params.ElasticNet
        schema = self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir = config.root_dir,
            test_data_path=config.test_data_path,
            model_path=config.model_path,
            all_params=params,
            metric_file_name=config.metric_file_name,
            target_column=schema.name,
            mlflow_uri="https://dagshub.com/atharvasamel/End-to-End-MLProject-using-MLflow-and-EC2-Insance.mlflow"
        )

        return model_evaluation_config

In [48]:
import os
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn
import numpy as np
import joblib 

In [49]:
class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config
    
    def eval_metrics(self, actual, pred):
        rmse = np.sqrt(mean_squared_error(actual,pred))
        mae = mean_absolute_error(actual, pred)
        r2 = r2_score(actual,pred)
        return rmse, mae, r2
    
    def log_into_mlflow(self):
        test_data = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)

        test_x = test_data.drop([self.config.target_column], axis = 1)
        test_y = test_data[[self.config.target_column]]

        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

        with mlflow.start_run():

            predicted_quantites = model.predict(test_x)

            (rsme, mae, r2) = self.eval_metrics(test_y, predicted_quantites)

            scores = {"rsme" : rsme, "mae" : mae, "r2" : r2}
            save_json(path = Path(self.config.metric_file_name), data = scores)

            # Model registry does not work with file store
            if tracking_url_type_store != "file":

                # Register the model
                # There are other ways to use the Model Registry, which depends on the use case,
                # please refer to the doc for more information:
                # https://mlflow.org/docs/latest/model-registry.html#api-workflow
                mlflow.sklearn.log_model(model, "model", registered_model_name="ElasticnetModel")
            else:
                mlflow.sklearn.log_model(model, "model")

In [50]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation_config = ModelEvaluation(config = model_evaluation_config)
    model_evaluation_config.log_into_mlflow()
except Exception as e:
    raise e

[2024-03-20 10:21:33,873: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-03-20 10:21:33,884: INFO: common: yaml file: params.yaml loaded successfully]
[2024-03-20 10:21:33,903: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-03-20 10:21:33,905: INFO: common: created directory at: artifacts]
[2024-03-20 10:21:33,911: INFO: common: created directory at: artifacts/model_evaluation]


[2024-03-20 10:21:34,201: INFO: common: json file saved at: artifacts\model_evaluation\metrics.json]
